In [1]:
import torch
from transformers import (GPT2Tokenizer, 
                          GPT2LMHeadModel, 
                          Trainer, 
                          TrainingArguments)
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType

/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#GPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#GPT-2 모델, 토크나이저 로드
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# EOS token 확인
print(f"EOS token: {tokenizer.eos_token}, EOS token ID: {tokenizer.eos_token_id}")

# pad_token 추가 (eos_token을 pad_token으로 사용)
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
# EOS token 확인
print(f"EOS token: {tokenizer.eos_token}, EOS token ID: {tokenizer.eos_token_id}")


EOS token: <|endoftext|>, EOS token ID: 50256
EOS token: <|endoftext|>, EOS token ID: 50256


/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
#dataset load 
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
eval_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="validation")

In [5]:


# 데이터셋 전처리 (토큰화 + labels 추가)
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = inputs["input_ids"].clone()  # labels에 input_ids를 복사하여 사용
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["text"])
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=["text"])


Map: 100%|██████████| 3760/3760 [00:00<00:00, 5231.66 examples/s]


# LoRA 적용

In [15]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=16, #학습률 스케일링 
    lora_dropout=0.2, 
    target_modules=["c_attn"],  # attention 레이어에 LoRA적용 
    # Q,K,V를 다르게 적용하고자 하여도 Hugging face에서 하나의 모듈로 통합되어서 개별 적용 못함. 
    task_type=TaskType.CAUSAL_LM #언어 모델링 태스크 
)

In [16]:
# gpt-2 모델 로드 후 모델에 LoRA적용 
lora_model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
lora_model = get_peft_model(lora_model, lora_config)


# pad_token이 추가된 후 토큰 임베딩 크기 재설정
lora_model.resize_token_embeddings(len(tokenizer))

/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Embedding(50257, 768)

In [17]:
# loss 반환시 오류가 발생하므로 Trainer에 compute_loss() 메서드를 재정의 

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs= False):
        #labels를 input에서 가져옴
        labels = inputs.get("labels")

        #inputs에서 labels를 제거하고 나머지를 모델에 전달
        outputs=model(**inputs)

        #model에서 logits를 반환
        logits = outputs.get("logits")
        # Shifted logits and labels로 손실 계산 (Causal Language Modeling Loss)
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        
        # CrossEntropyLoss 함수로 손실 계산
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

In [18]:
training_args_lora = TrainingArguments(
    output_dir="./lora_results",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    save_total_limit=2,
    learning_rate=5e-4,
    fp16=True,  # GPU의 half precision 사용
    report_to="none"  # 원하지 않는 리포팅 툴로의 연결을 방지
)

/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# LoRA 모델 트레이너 정의
lora_trainer = Trainer(
    model=lora_model,
    args=training_args_lora,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_eval_dataset,  # 평가용 데이터셋

)

/home/park/anaconda3/envs/NLtrans/lib/python3.11/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [20]:
# LoRA 모델 학습
lora_trainer.train()

  1%|          | 500/45900 [00:08<12:25, 60.87it/s]

{'loss': 1.6042, 'grad_norm': 0.5960968732833862, 'learning_rate': 0.0004945751633986928, 'epoch': 0.05}


                                                   
  1%|          | 500/45900 [00:14<12:25, 60.87it/s]

{'eval_loss': 1.3665893077850342, 'eval_runtime': 5.6658, 'eval_samples_per_second': 663.631, 'eval_steps_per_second': 82.954, 'epoch': 0.05}


  2%|▏         | 1000/45900 [00:22<12:21, 60.58it/s] 

...

                                                     
 99%|█████████▉| 45500/45900 [21:39<00:06, 60.10it/s]

{'eval_loss': 1.2904012203216553, 'eval_runtime': 5.6093, 'eval_samples_per_second': 670.319, 'eval_steps_per_second': 83.79, 'epoch': 4.96}


100%|██████████| 45900/45900 [21:47<00:00, 35.12it/s]

{'train_runtime': 1307.0626, 'train_samples_per_second': 140.46, 'train_steps_per_second': 35.117, 'train_loss': 1.3187543216613902, 'epoch': 5.0}


TrainOutput(global_step=45900, training_loss=1.3187543216613902, metrics={'train_runtime': 1307.0626, 'train_samples_per_second': 140.46, 'train_steps_per_second': 35.117, 'total_flos': 1.201344191004672e+16, 'train_loss': 1.3187543216613902, 'epoch': 5.0})

In [21]:
lora_model.save_pretrained("./lora_gpt2")
tokenizer.save_pretrained("./lora_gpt2")


('./lora_gpt2/tokenizer_config.json',
 './lora_gpt2/special_tokens_map.json',
 './lora_gpt2/vocab.json',
 './lora_gpt2/merges.txt',
 './lora_gpt2/added_tokens.json')

In [22]:
for name, module in lora_model.named_modules():
    print(name)


base_model
base_model.model
base_model.model.transformer
base_model.model.transformer.wte
base_model.model.transformer.wpe
base_model.model.transformer.drop
base_model.model.transformer.h
base_model.model.transformer.h.0
base_model.model.transformer.h.0.ln_1
base_model.model.transformer.h.0.attn
base_model.model.transformer.h.0.attn.c_attn
base_model.model.transformer.h.0.attn.c_attn.base_layer
base_model.model.transformer.h.0.attn.c_attn.lora_dropout
base_model.model.transformer.h.0.attn.c_attn.lora_dropout.default
base_model.model.transformer.h.0.attn.c_attn.lora_A
base_model.model.transformer.h.0.attn.c_attn.lora_A.default
base_model.model.transformer.h.0.attn.c_attn.lora_B
base_model.model.transformer.h.0.attn.c_attn.lora_B.default
base_model.model.transformer.h.0.attn.c_attn.lora_embedding_A
base_model.model.transformer.h.0.attn.c_attn.lora_embedding_B
base_model.model.transformer.h.0.attn.c_attn.lora_magnitude_vector
base_model.model.transformer.h.0.attn.c_proj
base_model.model.

In [23]:

# LoRA 모델 평가
lora_results = lora_trainer.evaluate()
print(f"LoRA Model Results: {lora_results}")

100%|██████████| 470/470 [00:05<00:00, 83.34it/s]

LoRA Model Results: {'eval_loss': 1.2904026508331299, 'eval_runtime': 5.6514, 'eval_samples_per_second': 665.328, 'eval_steps_per_second': 83.166, 'epoch': 5.0}


In [31]:
# 전체 파라미터 수 계산 함수
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# 기존 GPT-2 모델 로드 및 파라미터 수 계산
baseline_model = GPT2LMHeadModel.from_pretrained(model_name).to(device)  # 기존 GPT-2 모델을 GPU로 로드
baseline_params = count_parameters(baseline_model)
print(f"Baseline GPT-2 Model Parameters: {baseline_params}")

# LoRA가 적용된 GPT-2 모델 파라미터 수 계산
lora_model = lora_model.to(device)  # LoRA 모델을 GPU로 이동
lora_params = count_parameters(lora_model)
print(f"LoRA GPT-2 Model Parameters: {lora_params}")

# 파라미터 수 비교
print("Parameter Count Comparison:")
print(f"Baseline GPT-2 Model: {baseline_params} parameters")
print(f"LoRA GPT-2 Model: {lora_params} parameters")


Baseline GPT-2 Model Parameters: 124439808
LoRA GPT-2 Model Parameters: 147456
Parameter Count Comparison:
Baseline GPT-2 Model: 124439808 parameters
LoRA GPT-2 Model: 147456 parameters


In [32]:
print(f"Pad token: {tokenizer.pad_token}, Pad token ID: {tokenizer.pad_token_id}")

Pad token: <|endoftext|>, Pad token ID: 50256


In [33]:
from datasets import load_metric
import evaluate

In [34]:
# 평가 메트릭 로드 (
bleu_metric = evaluate.load("bleu")
meteor_metric = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /home/park/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/park/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/park/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [40]:
# 참조 문장 설정 
references = [
    ["The cat is on the mat.", "A cat is sitting on the mat.", "There is a cat on the mat.", "The feline is resting on the rug."],
    ["The feline lies on the carpet.", "The cat rests on the mat.", "A small cat is sitting on the mat."],
    ["A cat is on the floor.", "The mat is where the cat lies.", "The cat is lying on the floor mat."]
]



In [41]:
inputs = tokenizer("The cat is", return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
attention_mask = tokenizer("The cat is", return_tensors="pt", padding=True, truncation=True).attention_mask.to(device)

# 1. GPT-2 모델로 텍스트 생성 (Beam Search 및 텍스트 길이 늘리기)
gpt2_generated_outputs = baseline_model.generate(
    inputs, 
    max_length=100,  # 텍스트 길이 늘림
    attention_mask=attention_mask, 
    pad_token_id=tokenizer.pad_token_id, 
    num_beams=5,  # Beam Search 적용
    early_stopping=True  # 적절한 위치에서 멈춤
)
gpt2_generated_text = tokenizer.decode(gpt2_generated_outputs[0], skip_special_tokens=True)

# 2. LoRA 모델로 텍스트 생성 (Beam Search 및 텍스트 길이 늘리기)
lora_generated_outputs = lora_model.generate(
    inputs, 
    max_length=100,  # 텍스트 길이 늘림
    attention_mask=attention_mask, 
    pad_token_id=tokenizer.pad_token_id, 
    num_beams=5,  # Beam Search 적용
    early_stopping=True
)
lora_generated_text = tokenizer.decode(lora_generated_outputs[0], skip_special_tokens=True)

# BLEU 및 METEOR 점수 계산

# GPT-2 모델 BLEU 및 METEOR 점수 계산
gpt2_metrics = {
    "bleu": bleu_metric.compute(predictions=[gpt2_generated_text], references=[references])["bleu"],
    "meteor": meteor_metric.compute(predictions=[gpt2_generated_text], references=[references])["meteor"],
}

# LoRA 모델 BLEU 및 METEOR 점수 계산
lora_metrics = {
    "bleu": bleu_metric.compute(predictions=[lora_generated_text], references=[references])["bleu"],
    "meteor": meteor_metric.compute(predictions=[lora_generated_text], references=[references])["meteor"],
}

# 결과 출력
print("GPT-2 Model Evaluation:")
print(f"BLEU: {gpt2_metrics['bleu']}, METEOR: {gpt2_metrics['meteor']}")

print("LoRA Model Evaluation:")
print(f"BLEU: {lora_metrics['bleu']}, METEOR: {lora_metrics['meteor']}")

GPT-2 Model Evaluation:
BLEU: 0.0, METEOR: 0.2601304945054945
LoRA Model Evaluation:
BLEU: 0.0, METEOR: 0.1756440281030445
